# **Threads - Escalonamento e Sincronismo** 

### **Autoras** 👩‍💻

Desenvolvedores que contribuíram para a estruturação e desenvolvimento deste projeto:

- **Iza Lopes Ribeiro** - [Izalp](https://github.com/Izalp)
- **Wiliane Carolina Silva** - [wilicarol](https://github.com/wilicarol)

## **Simulação das modalidades**

In [32]:
import threading
import time
import random
from sports import sports


# Função para simular a execução do esporte
def sport(title, duration):
    start_time = time.perf_counter()  
    print(f'Modalidade {title:<25} | Início: {start_time:.3f} segundos')

    time.sleep(duration)  # Simula a execução do esporte pelo tempo de duração

    end_time = time.perf_counter() 
    print(f'Modalidade {title:<25} | Término: {end_time:.3f} segundos | Duração: {duration:.3f} segundos')
    return duration

## **Escalonamento FCFS (First-Come, First-Served)**

**Descrição:** 
- Threads são executadas na ordem de chegada, sem interrupções
- Pode causar longas esperas se uma thread demorar muito

In [2]:
def escalonamento_fcfs(pending_sports):
    total_duration = 0  # Total de duração dos esportes
    total_wait_time = 0  # Total de tempo de espera
    previous_end_time = 0  # Tempo em que o último esporte terminou
    wait_times = []  # Lista para armazenar os tempos de espera de cada esporte

    for title, duration in pending_sports:
        # Cálculo do tempo de espera
        wait_time = previous_end_time  # O tempo de espera é o tempo que o último esporte levou para terminar
        wait_times.append(wait_time)  # Armazena o tempo de espera para cada esporte
        total_wait_time += wait_time
        
        # Inicia o esporte e aguarda sua conclusão
        t = threading.Thread(target=sport, args=(title, duration))
        t.start()
        t.join()  # Executa o esporte até completar

        # Atualiza os totais
        total_duration += duration
        previous_end_time += duration  # Atualiza o tempo em que o próximo esporte começa a ser executado

    return total_duration, total_wait_time, len(pending_sports), wait_times  

In [3]:
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    total_duration, total_wait_time, num_sports, wait_times = escalonamento_fcfs(pending_sports)

    # Calcula a média de duração e de tempo de espera
    avg_duration_time = total_duration / num_sports if num_sports else 0
    avg_wait_time = total_wait_time / num_sports if num_sports else 0
    
    print('\n')
    # Exibe os tempos de espera para cada esporte
    for i, (title, _) in enumerate(pending_sports):
        print(f'Modalidade {title:<25} | Tempo de Espera: {wait_times[i]:.3f} segundos')

    print(f'\nMédia de tempo de duração (FCFS): {avg_duration_time:.3f} segundos')
    print(f'Média de tempo de espera (FCFS): {avg_wait_time:.3f} segundos\n')

start_olympics_fcfs()


Os Jogos Olímpicos com FCFS foram iniciados!

Modalidade Breaking                  | Início: 2192424.990 segundos
Modalidade Breaking                  | Término: 2192429.941 segundos | Duração: 4.951 segundos
Modalidade Ciclismo de Pista         | Início: 2192429.948 segundos
Modalidade Ciclismo de Pista         | Término: 2192431.818 segundos | Duração: 1.869 segundos
Modalidade Saltos Ornamentais        | Início: 2192431.825 segundos
Modalidade Saltos Ornamentais        | Término: 2192432.369 segundos | Duração: 0.544 segundos
Modalidade Badminton                 | Início: 2192432.372 segundos
Modalidade Badminton                 | Término: 2192436.876 segundos | Duração: 4.504 segundos
Modalidade Tênis                     | Início: 2192436.883 segundos
Modalidade Tênis                     | Término: 2192438.202 segundos | Duração: 1.319 segundos
Modalidade Canoagem Slalom           | Início: 2192438.207 segundos
Modalidade Canoagem Slalom           | Término: 2192442.905 segundos |

## **Escalonamento Round Robin (RR)**

**Descrição:**
- Cada thread recebe um tempo fixo (quantum) para executar. Se não terminar, volta ao final da fila
- A performance depende do tamanho do quantum

In [4]:
def escalonamento_rr(pending_sports, lock, completed_sports, quantum=2):
    current_time = time.perf_counter()  # Momento inicial
    wait_times = []
    while pending_sports:
        for i in range(len(pending_sports)):
            title, duration, arrival_time = pending_sports.pop(0)
            effective_duration = min(duration, quantum)
            lock.acquire()  # Protege a seção crítica
            
            # Calcular tempo de espera antes da execução do esporte
            wait_time = current_time - arrival_time
            wait_times.append(wait_time)
            
            # Cria e executa o thread para o esporte com duração limitada pelo quantum
            t = threading.Thread(target=sport, args=(title, effective_duration))
            t.start()
            t.join()
            lock.release()  # Libera o lock

            # Atualiza o tempo atual após a execução
            current_time = time.perf_counter()

            # Se o esporte não terminou, retorna com o tempo restante
            if duration > quantum:
                pending_sports.append((title, duration - quantum, current_time))
            else:
                completed_sports.append((title, wait_time))
    
    return wait_times

In [5]:
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados
    lock = threading.Lock()
    completed_sports = []  # Lista de esportes concluídos e seus tempos de espera
    total_durations = []  # Lista para armazenar os tempos de duração

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias em segundos
        arrival_time = time.perf_counter()  # Tempo de chegada na fila

        pending_sports.append((title, duration, arrival_time))
        total_durations.append(duration)  # Adiciona a duração ao total

    wait_times = escalonamento_rr(pending_sports, lock, completed_sports)


    print('\n')
    # Exibe os tempos de espera para cada esporte
    for i, (title, _) in enumerate(completed_sports):
        print(f'Modalidade {title:<25} | Tempo de Espera: {wait_times[i]:.3f} segundos')

    # Calcula a média dos tempos de duração
    avg_duration = sum(total_durations) / len(total_durations) if total_durations else 0
    print(f'\nMédia de duração dos esportes: {avg_duration:.3f} segundos')

    # Calcula a média dos tempos de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'Média de tempo de espera (RR): {avg_wait_time:.3f} segundos\n')


# Inicia a simulação com Round Robin
start_olympics_rr()


Os Jogos Olímpicos com RR foram iniciados!

Modalidade Natação                   | Início: 2192565.665 segundos
Modalidade Natação                   | Término: 2192567.665 segundos | Duração: 2.000 segundos
Modalidade Badminton                 | Início: 2192567.673 segundos
Modalidade Badminton                 | Término: 2192569.674 segundos | Duração: 2.000 segundos
Modalidade Skate                     | Início: 2192569.676 segundos
Modalidade Skate                     | Término: 2192569.817 segundos | Duração: 0.141 segundos
Modalidade Futebol                   | Início: 2192569.819 segundos
Modalidade Futebol                   | Término: 2192571.820 segundos | Duração: 2.000 segundos
Modalidade Ciclismo Bmx Freestyle    | Início: 2192571.822 segundos
Modalidade Ciclismo Bmx Freestyle    | Término: 2192572.644 segundos | Duração: 0.822 segundos
Modalidade Tiro com Arco             | Início: 2192572.645 segundos
Modalidade Tiro com Arco             | Término: 2192572.678 segundos | D

## **Implementação dos Mecanismos de Sincronização**

###  **Semáforo Binário**

**Descrição:** Controla o acesso de uma thread por vez a uma seção crítica, funcionando como um “lock” com valores 0 e 1.

In [33]:
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução')
        sport(title, duration)
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso')

In [15]:
def run_sports(sports, method):
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5)  
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time


binary_semaphore_time = run_sports(sports, sport_with_binary_semaphore)
print(f'\n[Semáforo Binário] Tempo total: {binary_semaphore_time:.3f} segundos\n')

[Semáforo Binário] Modalidade Atletismo aguardando acesso[Semáforo Binário] Modalidade Badminton aguardando acesso
[Semáforo Binário] Modalidade Badminton iniciou execução
Modalidade Badminton                 | Início: 2193214.986 segundos

[Semáforo Binário] Modalidade Basquete aguardando acesso
[Semáforo Binário] Modalidade Basquete 3x3 aguardando acesso
[Semáforo Binário] Modalidade Boxe aguardando acesso
[Semáforo Binário] Modalidade Breaking aguardando acesso
[Semáforo Binário] Modalidade Canoagem de Velocidade aguardando acesso
[Semáforo Binário] Modalidade Canoagem Slalom aguardando acesso
[Semáforo Binário] Modalidade Ciclismo Bmx Freestyle aguardando acesso
[Semáforo Binário] Modalidade Ciclismo Bmx Racing aguardando acesso
[Semáforo Binário] Modalidade Ciclismo de Estrada aguardando acesso
[Semáforo Binário] Modalidade Ciclismo de Pista aguardando acesso
[Semáforo Binário] Modalidade Ciclismo Mountain Bike aguardando acesso
[Semáforo Binário] Modalidade Escalada aguardando ac

### **Monitor**

**Descrição:** Estrutura que garante exclusão mútua e organiza o acesso de threads com métodos e variáveis de condição.

In [16]:
monitor_lock = threading.Lock()

def sport_with_monitor(title, duration):
    print(f'[Monitor] Modalidade {title} aguardando acesso')
    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução')
        sport(title, duration)
    print(f'[Monitor] Modalidade {title} liberou o acesso')


In [17]:
def run_sports(sports, method):
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5)  
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time


monitor_time = run_sports(sports, sport_with_monitor)
print(f'\n[Monitor] Tempo total: {monitor_time:.3f} segundos\n')

[Monitor] Modalidade Atletismo aguardando acesso
[Monitor] Modalidade Atletismo iniciou execução
Modalidade Atletismo                 | Início: 2193351.438 segundos
[Monitor] Modalidade Badminton aguardando acesso
[Monitor] Modalidade Basquete aguardando acesso
[Monitor] Modalidade Basquete 3x3 aguardando acesso
[Monitor] Modalidade Boxe aguardando acesso
[Monitor] Modalidade Breaking aguardando acesso
[Monitor] Modalidade Canoagem de Velocidade aguardando acesso
[Monitor] Modalidade Canoagem Slalom aguardando acesso
[Monitor] Modalidade Ciclismo Bmx Freestyle aguardando acesso
[Monitor] Modalidade Ciclismo Bmx Racing aguardando acesso
[Monitor] Modalidade Ciclismo de Estrada aguardando acesso
[Monitor] Modalidade Ciclismo de Pista aguardando acesso
[Monitor] Modalidade Ciclismo Mountain Bike aguardando acesso
[Monitor] Modalidade Escalada aguardando acesso
[Monitor] Modalidade Esgrima aguardando acesso
[Monitor] Modalidade Futebol aguardando acesso
[Monitor] Modalidade Ginástica Artís

### **Semáforo de Contagem**

**Descrição:** Permite acesso simultâneo de várias threads a um recurso até um limite definido pelo contador.

In [18]:
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução')
        sport(title, duration)
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso')

In [19]:
def run_sports(sports, method):
    threads = []
    start_time = time.perf_counter()
    
    for title in sports:
        duration = random.uniform((1/60),5) 
        thread = threading.Thread(target=method, args=(title, duration))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()
    
    end_time = time.perf_counter()
    return end_time - start_time


counting_semaphore_time = run_sports(sports, sport_with_counting_semaphore)
print(f'\n[Semáforo de Contagem] Tempo total: {counting_semaphore_time:.3f} segundos\n')

[Semáforo de Contagem] Modalidade Atletismo aguardando acesso
[Semáforo de Contagem] Modalidade Atletismo iniciou execução
Modalidade Atletismo                 | Início: 2193479.292 segundos
[Semáforo de Contagem] Modalidade Badminton aguardando acesso
[Semáforo de Contagem] Modalidade Badminton iniciou execução
Modalidade Badminton                 | Início: 2193479.292 segundos
[Semáforo de Contagem] Modalidade Basquete aguardando acesso
[Semáforo de Contagem] Modalidade Basquete 3x3 aguardando acesso
[Semáforo de Contagem] Modalidade Boxe aguardando acesso
[Semáforo de Contagem] Modalidade Breaking aguardando acesso
[Semáforo de Contagem] Modalidade Canoagem de Velocidade aguardando acesso
[Semáforo de Contagem] Modalidade Canoagem Slalom aguardando acesso
[Semáforo de Contagem] Modalidade Ciclismo Bmx Freestyle aguardando acesso
[Semáforo de Contagem] Modalidade Ciclismo Bmx Racing aguardando acesso
[Semáforo de Contagem] Modalidade Ciclismo de Estrada aguardando acesso
[Semáforo de

## **Escalonamento com Sincronismo**

### FCFS + Semáforo Binário

In [34]:
def fcfs_with_binary_semaphore(pending_sports, wait_times, duration_times):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        
        # Inicia a contagem do tempo de espera
        wait_start_time = time.perf_counter()  
        
        t = threading.Thread(target=sport_with_binary_semaphore, args=(title, duration))
        executed_today.append((t, title))
        t.start()
        t.join()  # Aguarda a finalização da thread
        
        # Após a execução, calcula o tempo de espera
        wait_end_time = time.perf_counter()
        wait_time = wait_end_time - wait_start_time  # O tempo que a thread esperou
        wait_times.append(wait_time)  # Adiciona o tempo de espera à lista
        duration_times.append(duration)  # Adiciona a duração à lista

    return executed_today

In [35]:
def start_fcfs_binary_semaphore():
    print('\nOs Jogos Olímpicos com FCFS + Semáforo Binário foram iniciados!\n')
    all_sports = sports.copy()  # Cópia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados
    wait_times = []  # Lista para armazenar os tempos de espera
    duration_times = []  # Lista para armazenar as durações dos esportes

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias em horas
        pending_sports.append((title, duration))

    fcfs_with_binary_semaphore(pending_sports, wait_times, duration_times)

    # Calcula a média do tempo de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    # Calcula a média do tempo de duração
    avg_duration_time = sum(duration_times) / len(duration_times) if duration_times else 0
    
    print(f'\nMédia de tempo de espera (FCFS + Semáforo Binário): {avg_wait_time:.3f} segundos')
    print(f'Média de tempo de duração (FCFS + Semáforo Binário): {avg_duration_time:.3f} segundos\n')

# Inicia a simulação com FCFS e semáforo binário
start_fcfs_binary_semaphore()


Os Jogos Olímpicos com FCFS foram iniciados!

[Semáforo Binário] Modalidade Tiro com Arco aguardando acesso
[Semáforo Binário] Modalidade Tiro com Arco iniciou execução
Modalidade Tiro com Arco             | Início: 2194728.853 segundos
Modalidade Tiro com Arco             | Término: 2194731.116 segundos | Duração: 2.263 segundos
[Semáforo Binário] Modalidade Tiro com Arco liberou o acesso
[Semáforo Binário] Modalidade Remo aguardando acesso
[Semáforo Binário] Modalidade Remo iniciou execução
Modalidade Remo                      | Início: 2194731.119 segundos
Modalidade Remo                      | Término: 2194735.574 segundos | Duração: 4.454 segundos
[Semáforo Binário] Modalidade Remo liberou o acesso
[Semáforo Binário] Modalidade Ginástica Rítmica aguardando acesso
[Semáforo Binário] Modalidade Ginástica Rítmica iniciou execução
Modalidade Ginástica Rítmica         | Início: 2194735.581 segundos
Modalidade Ginástica Rítmica         | Término: 2194739.895 segundos | Duração: 4.314 s

### RR + Semáforo de Contagem

In [14]:
lock = threading.Lock()
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours, wait_times):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    start_wait_time = time.perf_counter() 

    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration 

    end_wait_time = time.perf_counter()  
    wait_time = end_wait_time - start_wait_time  # Calcula o tempo de espera total
    
    with lock:
        wait_times.append(wait_time) 
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')

In [15]:
# Escalonamento RR com semáforo de contagem
def rr_with_counting_semaphore(pending_sports, daily_total_hours, wait_times, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_counting_semaphore, args=(title, min(duration, quantum), daily_total_hours, wait_times))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

In [16]:
def start_rr_counting_semaphore():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior
    daily_total_hours = [0]  # Lista para manter o total de horas do dia
    wait_times = []  # Lista para armazenar os tempos de espera

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    rr_with_counting_semaphore(pending_sports, daily_total_hours, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (RR + Semáforo de Contagem): {avg_wait_time:.3f} segundos\n')    


# Inicia a simulação com RR e semáforo de contagem
start_rr_counting_semaphore()



Os Jogos Olímpicos com RR foram iniciados!

[Semáforo de Contagem] Modalidade Tiro Esportivo aguardando acesso.
[Semáforo de Contagem] Modalidade Tiro Esportivo iniciou execução.
[Semáforo de Contagem] Modalidade Tiro Esportivo liberou o acesso.
[Semáforo de Contagem] Modalidade Vôlei aguardando acesso.
[Semáforo de Contagem] Modalidade Vôlei iniciou execução.
[Semáforo de Contagem] Modalidade Vôlei liberou o acesso.
[Semáforo de Contagem] Modalidade Tênis de Mesa aguardando acesso.
[Semáforo de Contagem] Modalidade Tênis de Mesa iniciou execução.
[Semáforo de Contagem] Modalidade Tênis de Mesa liberou o acesso.
[Semáforo de Contagem] Modalidade Nado Artístico aguardando acesso.
[Semáforo de Contagem] Modalidade Nado Artístico iniciou execução.
[Semáforo de Contagem] Modalidade Nado Artístico liberou o acesso.
[Semáforo de Contagem] Modalidade Esgrima aguardando acesso.
[Semáforo de Contagem] Modalidade Esgrima iniciou execução.
[Semáforo de Contagem] Modalidade Esgrima liberou o aces

### RR + Monitor

In [20]:
monitor_lock = threading.Lock()
num_sports = 5            # número de threads simultâneas para sincronizar
start_barrier = threading.Barrier(num_sports)

def sport_with_monitor(title, duration, daily_total_hours, wait_times):
    print(f'[Monitor] Modalidade {title} aguardando sincronismo.')
    start_wait_time = time.perf_counter() 

    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução.')
        time.sleep(duration)  # Simula o tempo de execução do esporte
        with lock:
            daily_total_hours[0] += duration

    end_wait_time = time.perf_counter()  
    wait_time = end_wait_time - start_wait_time  # Calcula o tempo de espera total
    
    with lock:
        wait_times.append(wait_time) 
    print(f'[Monitor] Modalidade {title} liberou o acesso.')

In [21]:
# Escalonamento RR com monitor
def rr_with_monitor(pending_sports, daily_total_hours, wait_times, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_monitor, args=(title, min(duration, quantum), daily_total_hours, wait_times))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

In [22]:
def start_rr_monitor():
    print('\nOs Jogos Olímpicos com RR foram iniciados!\n')

    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior
    daily_total_hours = [0]  # Lista para manter o total de horas do dia
    wait_times = []  # Lista para armazenar os tempos de espera

    while all_sports:
        title = random.choice(all_sports)
        all_sports.remove(title)
        duration = random.uniform((1/60), 5)  # Durações aleatórias

        pending_sports.append((title, duration))

    rr_with_monitor(pending_sports, daily_total_hours, wait_times)

    # Calcula a média de espera
    avg_wait_time = sum(wait_times) / len(wait_times) if wait_times else 0
    print(f'\nMédia de tempo de espera (RR + Monitor): {avg_wait_time:.3f} segundos\n')    


# Inicia a simulação com RR e monitor
start_rr_monitor()


Os Jogos Olímpicos com RR foram iniciados!

[Monitor] Modalidade Boxe aguardando sincronismo.
[Monitor] Modalidade Boxe iniciou execução.
[Monitor] Modalidade Boxe liberou o acesso.
[Monitor] Modalidade Vôlei de Praia aguardando sincronismo.
[Monitor] Modalidade Vôlei de Praia iniciou execução.
[Monitor] Modalidade Vôlei de Praia liberou o acesso.
[Monitor] Modalidade Tiro Esportivo aguardando sincronismo.
[Monitor] Modalidade Tiro Esportivo iniciou execução.
[Monitor] Modalidade Tiro Esportivo liberou o acesso.
[Monitor] Modalidade Triatlo aguardando sincronismo.
[Monitor] Modalidade Triatlo iniciou execução.
[Monitor] Modalidade Triatlo liberou o acesso.
[Monitor] Modalidade Vela aguardando sincronismo.
[Monitor] Modalidade Vela iniciou execução.
[Monitor] Modalidade Vela liberou o acesso.
[Monitor] Modalidade Ciclismo de Estrada aguardando sincronismo.
[Monitor] Modalidade Ciclismo de Estrada iniciou execução.
[Monitor] Modalidade Ciclismo de Estrada liberou o acesso.
[Monitor] Mod

#### Comparação de resultado: 

FCFS será mais simples, com as modalidades executando uma por vez. O tempo total de execução será maior, pois nenhuma modalidade é interrompida.

Round Robin com semáforo de contagem permite execuções simultâneas, mas interrompe modalidades que ultrapassam o quantum, o que pode melhorar a eficiência em termos de utilização de recursos.

OBS.: O monitor não foi utilizado diretamente no código anterior porque, na prática, ele é bastante semelhante ao semáforo binário.